Рисует рандомные точки.. Уже неплохо Будут звездами по умолчанию Там размебермся как потом сохранять как карту и подгружать.

In [1]:
import sys
from PyQt5 import QtCore, QtGui, QtWidgets, uic
from PyQt5.QtCore import Qt


class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()

        self.label = QtWidgets.QLabel()
        canvas = QtGui.QPixmap(400, 300)
        canvas.fill(Qt.white)
        self.label.setPixmap(canvas)
        self.setCentralWidget(self.label)
        self.draw_something()

    def draw_something(self):
        painter = QtGui.QPainter(self.label.pixmap())
        painter.drawLine(10, 10, 300, 200)
        painter.end()

    def draw_something(self):
        from random import randint, choice
        colors = ['#FFD141', '#376F9F', '#0D1F2D', '#E9EBEF', '#EB5160']

        painter = QtGui.QPainter(self.label.pixmap())
        pen = QtGui.QPen()
        pen.setWidth(3)
        painter.setPen(pen)

        for n in range(10000):
            # pen = painter.pen() you could get the active pen here
            pen.setColor(QtGui.QColor(choice(colors)))
            painter.setPen(pen)
            painter.drawPoint(
                200+randint(-100, 100),  # x
                150+randint(-100, 100)   # y
                )
        painter.end()

app = QtWidgets.QApplication(sys.argv)
window = MainWindow()
window.show()
app.exec_()

0

Бесполезный пока круг.. (Но позже возможно будет полезным)

In [2]:
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget
from PyQt5.QtGui import QIcon, QPainter, QPen, QPixmap, QPainterPath
from PyQt5.QtCore import Qt, QRectF, QPoint, QPointF, QVariantAnimation
from PyQt5 import QtGui
from PyQt5.QtGui import QPainter, QPen, QBrush

class CirclingWidget(QWidget):
    offset = QPoint()
    def __init__(self, diameter=1, relative=True, sprite=None, 
            anglesPerSecond=360, clockwise=True, offset=None, alignment=None):
        super().__init__()
        self.diameter = diameter
        self.relative = relative
        self.clockwise = clockwise
        self.setSprite(sprite, offset or alignment)
        self.animation = QVariantAnimation(startValue=0., endValue=1., 
            duration=360000 / anglesPerSecond, loopCount=-1)
        self.animation.valueChanged.connect(self.update)

    def setSprite(self, sprite, arg):
        if isinstance(sprite, str):
            sprite = QPixmap(sprite)
        if isinstance(sprite, QPixmap) and not sprite.isNull():
            self.sprite = sprite
            if isinstance(arg, Qt.AlignmentFlag):
                self.setAlignment(arg)
            else:
                self.setOffset(arg)
        else:
            self.sprite = None
        self.update()

    def setAlignment(self, alignment=None):
        if self.sprite:
            x = y = 0
            if alignment is not None:
                if alignment & Qt.AlignHCenter:
                    x = -self.sprite.width() / 2
                elif alignment & Qt.AlignRight:
                    x = -self.sprite.width()
                if alignment & Qt.AlignVCenter:
                    y = -self.sprite.height() / 2
                elif alignment & Qt.AlignBottom:
                    y = -self.sprite.height()
            self.offset = QPointF(x, y)
        self.update()

    def setOffset(self, offset=None):
        if self.sprite:
            x = y = 0
            if isinstance(offset, int):
                x = y = offset
            elif isinstance(offset, float):
                x = self.sprite.width() * offset
                y = self.sprite.height() * offset
            elif isinstance(offset, (QPoint, QPointF)):
                x = offset.x()
                y = offset.y()
            elif isinstance(offset, (tuple, list)):
                x, y = offset
            self.offset = QPointF(x, y)
        self.update()

    def setAnglesPerSecond(self, ratio):
        self.animation.setDuration(360000 / ratio)

    def setClockwise(self, clockwise=True):
        if self.clockwise != clockwise:
            self.clockwise = clockwise
            self.animation.setCurrentTime(
                self.animation.duration() - self.animation.currentTime())
            self.update()

    def start(self, angle=0):
        self.animation.start()
        # the angle is considered clockwise, starting from 12h
        start = (angle - 90) / 360 % 1
        self.animation.setCurrentTime(start * self.animation.duration())
        self.update()

    def stop(self):
        self.animation.stop()
        self.update()

    def paintEvent(self, event):
        painter  = QPainter(self)
        painter.setRenderHints(painter.Antialiasing)
        painter.setPen(QPen(Qt.black, 5, Qt.SolidLine))

        size = self.diameter
        if self.relative:
            size *= min(self.width(), self.height())

        rect = QRectF(0, 0, size, size)
        rect.moveCenter(QRectF(self.rect()).center())

        path = QPainterPath()
        path.addEllipse(rect)
        painter.drawPath(path)

        if self.sprite:
            pos = self.animation.currentValue()
            if not self.clockwise:
                pos = 1 - pos
            pos = path.pointAtPercent(pos)
            painter.drawPixmap(pos + self.offset, self.sprite)


class Window(QMainWindow):
    def __init__(self):
        super().__init__()

        self.title = "Drawing Ellipse"
        self.top = 200
        self.left = 200
        self._width = 800
        self._height = 800
        self.InitWindow()


    def InitWindow(self):
        self.setWindowIcon(QIcon("icon.png"))
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self._width, self._height)

        self.circleWidget = CirclingWidget(.5, sprite='image.png', 
            anglesPerSecond=60, alignment=Qt.AlignCenter)
        self.setCentralWidget(self.circleWidget)
        self.circleWidget.start(90)

        self.show()


if __name__ == '__main__':
    import sys
    App = QApplication(sys.argv)
    window = Window()
    sys.exit(App.exec())

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_4620\226764062.py:16: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.animation = QVariantAnimation(startValue=0., endValue=1.,
C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_4620\226764062.py:78: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.animation.setCurrentTime(start * self.animation.duration())


SystemExit: 0

c:\Users\Artur.Abaidulov\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Радарная карта.. Посмотрим может быть, что нибудь придумаю для нее.

In [1]:
import sys
import random
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.Qt import *
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QSizePolicy

#from PyQt5 import QTimer

class GraphInit(FigureCanvas):
    def __init__(self, parent=None):
        fig = Figure()
        self.axes = fig.add_subplot(111, projection="polar")
        self.compute_initial_figure()

        FigureCanvas.__init__(self, fig)
        self.setParent(parent)

        FigureCanvas.setSizePolicy(self, QSizePolicy.Expanding, QSizePolicy.Expanding)
        FigureCanvas.updateGeometry(self)


class GraphPopulate(GraphInit):
    def compute_initial_figure(self):
        self.r =[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
        angles = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        self.axes.plot(angles, self.r, 'bo', ms=5)
        
    def _polar(self):
        self.axes.clear()
        for i in range(20):
            angles = random.choices(range(50), k=10)
            ms = random.randint(1, 7)
            self.axes.plot(angles, self.r, 'bo', ms=ms)
        self.draw()


class GUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.centralwidget = QtWidgets.QWidget()
        self.setCentralWidget(self.centralwidget)
        self.setMinimumSize(QtCore.QSize(270, 270))
        
        self.widget = QtWidgets.QWidget(self.centralwidget)
        self.sc = GraphPopulate(self.widget)
        self.gridLayout = QtWidgets.QGridLayout(self.centralwidget)
        self.gridLayout.addWidget(self.sc)
        
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update_polar)
        self.timer.start(1000)
        
    def update_polar(self):
        self.sc._polar()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = GUI()
    #w.resize(600, 600)
    w.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\Artur.Abaidulov\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Нашел отдельно движущуюся линию Уже хорошо. Будет линией линейки от планеты до планеты..

In [1]:
import sys
from PyQt5.QtWidgets import QMainWindow, QPushButton, QApplication
from PyQt5.QtWidgets import QApplication, QGraphicsView, QGraphicsScene, QGraphicsEllipseItem



from PyQt5.QtCore import QSize, Qt, QLine, QPoint
from PyQt5.QtGui import QPainter, QPen, QCursor

class GraphicView(QGraphicsView):
    def __init__(self):
        super().__init__()

        self.scene = QGraphicsScene()
        self.setScene(self.scene)       
        self.setSceneRect(300, 300, 1200, 1000)

class MainWindow(QMainWindow):
    def __init__(self):
        QMainWindow.__init__(self)

        # set window size
        self.setMinimumSize(QSize(500, 500))

        # create main button
        self.pybutton = QPushButton('button', self)
        self.pybutton.resize(100, 100)
        
        # Crate second button
        self.pybutton2 = QPushButton('b2', self)
        self.pybutton2.resize(10, 10)
        self.space = GraphicView()
        # inicialise line 
        self.line = QLine()
        #self.statusBar()
        self.space

    def draw_line(self,x,y,x1=0,y1=0):
        #sender = self.sender()
        pos=QPoint(x,y)
        #self.statusBar().showMessage(sender.text() + ' was pressed')
        self.line = QLine(QPoint(x1,y1),pos)
        self.update()

    def paintEvent(self, event):
        # QMainWindow.paintEvent(self, event)

        qp = QPainter()
        qp.begin(self)
        pen = QPen(Qt.red, 2, Qt.SolidLine)
        qp.setPen(pen)
        qp.drawLine(self.line)
        qp.end()

    #def mousePressEvent(self, event):
     #   self.pybutton.move(event.x(), event.y())

    def mouseMoveEvent(self,vent):
        self.pybutton.move(vent.x(),vent.y())
        self.pybutton2.click()
        self.draw_line(vent.x(),vent.y())



if __name__ == "__main__":
    app = QApplication(sys.argv)
    mainWin = MainWindow()
    mainWin.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\Artur.Abaidulov\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


https://www.pythonguis.com/tutorials/pyqt-layouts/
Очень полезный рессурс .. Тут описано как делать слои .. Думаю карта будет у меня прорисована слоями в зависимости от того, кому на какой слой можно смотреть.


https://www.pythonguis.com/tutorials/qscrollarea/
scroll in window  (Если карта большая то скрол понадобится надеюсь получится научить этому окно.)


Полезная штука для рисования карты. Думаю подойдет (поиграю посмотрим.)

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QGraphicsView, QGraphicsScene, QGraphicsEllipseItem
from PyQt5.QtCore import Qt, QPointF


class MovingObject(QGraphicsEllipseItem):
    def __init__(self, x, y, r):
        super().__init__(0, 0, r, r)
        self.setPos(x, y)
        self.setBrush(Qt.blue)
        self.setAcceptHoverEvents(True)

    # mouse hover event
    def hoverEnterEvent(self, event):
        app.instance().setOverrideCursor(Qt.OpenHandCursor)

    def hoverLeaveEvent(self, event):
        app.instance().restoreOverrideCursor()

    # mouse click event
    def mousePressEvent(self, event):
        pass

    def mouseMoveEvent(self, event):
        orig_cursor_position = event.lastScenePos()
        updated_cursor_position = event.scenePos()

        orig_position = self.scenePos()

        updated_cursor_x = updated_cursor_position.x() - orig_cursor_position.x() + orig_position.x()
        updated_cursor_y = updated_cursor_position.y() - orig_cursor_position.y() + orig_position.y()
        self.setPos(QPointF(updated_cursor_x, updated_cursor_y))
        self.draw_line(event.x(),event.y())

    def mouseReleaseEvent(self, event):
        print('x: {0}, y: {1}'.format(self.pos().x(), self.pos().y()))

    def draw_line(self,x,y,x1=0,y1=0):
        #sender = self.sender()
        pos=QPoint(x,y)
        #self.statusBar().showMessage(sender.text() + ' was pressed')
        self.line = QLine(QPoint(x1,y1),pos)
        self.update()

    def paintEvent(self, event):
        # QMainWindow.paintEvent(self, event)

        qp = QPainter()
        qp.begin(self)
        pen = QPen(Qt.red, 2, Qt.SolidLine)
        qp.setPen(pen)
        qp.drawLine(self.line)
        qp.end()    

class GraphicView(QGraphicsView):
    def __init__(self):
        super().__init__()

        self.scene = QGraphicsScene()
        self.setScene(self.scene)       
        self.setSceneRect(0, 0, 1200, 1000)

        self.moveObject = MovingObject(50, 50, 40)
        #self.moveObject2 = MovingObject(100, 100, 100)
        self.scene.addItem(self.moveObject)
        #self.scene.addItem(self.moveObject2)

    


app = QApplication(sys.argv)
view = GraphicView()
view.show()
sys.exit(app.exec_())

SystemExit: 0

c:\Users\Artur.Abaidulov\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


ПРоверяю код окна в окне    

Ураааа добился того, что я могу размещать в нужном месте нужное окно (только с сайзингом окна пока туговато, но думаю не страшно, когда все заполнится.)
Я пока крайней мере могу разместить каркас в нужных местах А там видно будет.

In [2]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *


import sys
class MyView(QGraphicsView):
    # Olaf -- Данная функция найдена на просторах интернета, и позволяет сцене прокручиваться средней кнопкой мыши если поле не влазает 
    # Я рад, теперь осталось набрать объектов в поле.
    def mousePressEvent(self, event):
        if event.button() == Qt.MidButton: # or Qt.MiddleButton
            self.__prevMousePos = event.pos()
        else:
            super(MyView, self).mousePressEvent(event)

    def mouseMoveEvent(self, event):
        if event.buttons() == Qt.MidButton: # or Qt.MiddleButton
            offset = self.__prevMousePos - event.pos()
            self.__prevMousePos = event.pos()

            self.verticalScrollBar().setValue(self.verticalScrollBar().value() + offset.y())
            self.horizontalScrollBar().setValue(self.horizontalScrollBar().value() + offset.x())
        else:
            super(MyView, self).mouseMoveEvent(event)


class DragButton(QPushButton):

    def mouseMoveEvent(self, e):

        if e.buttons() == Qt.LeftButton:
            drag = QDrag(self)
            mime = QMimeData()
            drag.setMimeData(mime)
            drag.exec_(Qt.MoveAction)

class GraphicView(QGraphicsView):
    def __init__(self):
        super().__init__()

        self.scene = QGraphicsScene(self)
          
        self.sceneRect().width(30)
        self.sceneRect().height(30)
        #self.setScene(self.scene)     

        rect_item = self.scene.addRect(QRectF(5, 5, 50, 50))
        rect_item.setPen(QPen(QColor("green")))


class Color(QWidget):

    def __init__(self, color):
        super(Color, self).__init__()
        self.setAutoFillBackground(True)

        palette = self.palette()
        palette.setColor(QPalette.Window, QColor(color))
        self.setPalette(palette)

class MainApp(QMainWindow):
    """This is the class of the MainApp GUI system"""
    def __init__(self):
        """Constructor method that inherits methods from QWidgets"""
        super().__init__()
        self.initUI()
    def dragEnterEvent(self, e):
        e.accept()

    def dropEvent(self, e):
        pos = e.pos()
        widget = e.source()

        for n in range(self.blayout.count()):
            # Get the widget at each index in turn.
            w = self.blayout.itemAt(n).widget()
            if pos.x() < w.x() + w.size().width() // 2:
                # We didn't drag past this widget.
                # insert to the left of it.
                self.blayout.insertWidget(n-1, widget)
                break

        e.accept()

    def initUI(self):
        """This method creates our GUI"""
        
        self.setWindowTitle("My App")
        

        layout_grid_main = QGridLayout()
        


        scene = QGraphicsScene()
        scene.setSceneRect(0, 0, 2700, 2700)
        scene.views
        
        model1 = MyView(scene)
        model2 = MyView(scene)
        model2.setStyleSheet("background:black;")
        model2.rubberBandRect()
        #model1.setDragEnabled(True)
  

        #model.scene = scen
        layout_grid_main.addWidget(QLabel("1"), 0, 0)
        layout_grid_main.addWidget(model2, 0, 1, 1, 3)
        layout_grid_main.addWidget(QLabel("3"), 1, 0, 1, 2)
        layout_grid_main.addWidget(model1, 1, 2, 1, 2)

        qsrand(QTime.currentTime().msec())

        widget = QWidget()
        widget.setLayout(layout_grid_main)
        self.setCentralWidget(widget)
        self.show()
if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = MainApp()
    sys.exit(app.exec_())

NameError: name 'QtWidgets' is not defined

AttributeError: 'Scene' object has no attribute 'currentQRubberBand'

AttributeError: 'Scene' object has no attribute 'currentQRubberBand'

AttributeError: 'Scene' object has no attribute 'currentQRubberBand'

AttributeError: 'Scene' object has no attribute 'currentQRubberBand'

...

NameError: name 'QtWidgets' is not defined

AttributeError: 'Scene' object has no attribute 'currentQRubberBand'

AttributeError: 'Scene' object has no attribute 'currentQRubberBand'

SystemExit: 0

c:\Users\Artur.Abaidulov\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
